In [1]:
# CIFAR 10 dataset - Numpy
import os
import platform
import pickle

import numpy as np
from numpy import asarray
from numpy import save
from numpy import load

import torch  
import torch.nn as nn
import torch.optim as optim

In [2]:
class VGG_11(nn.Module):
    
    def __init__(self):
        super(VGG_11,self).__init__()
        
        # Setting up the layers of Convolutional neural network
        in_size = 3                 # number of channel in the input image
    
        hid1_size = 64              # no of output channel from first CNN layer

        hid2_size = 128             # no of output channel from second CNN layer

        hid3_size = 256             # no of output channel from third CNN layer

        hid4_size = 512             # no of output channel from forth CNN layer

        classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

        out_size = len(classes)     # no of categories in the dataset

        k_conv_size = 3             # 3x3 convolutional kernel

        conv_stride = 1             # conv stride 1

        conv_pad = 1                # conv padding 1

        maxpool_kernel = 2          # maxpool layer kernel size 2 x 2

        maxpool_stride = 2          # maxpool layer stride 2

        self.convLayer = nn.Sequential(
            nn.Conv2d(in_size, hid1_size, k_conv_size, stride=conv_stride, padding=conv_pad),    # conv layer
            nn.BatchNorm2d(hid1_size),
            #nn.LocalResponseNorm(64),
            nn.ReLU(),                              # Activation layer
            nn.MaxPool2d(kernel_size=maxpool_kernel,stride=maxpool_stride),
            
            nn.Conv2d(hid1_size,hid2_size,k_conv_size, stride=conv_stride, padding=conv_pad),
            nn.BatchNorm2d(hid2_size),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=maxpool_kernel,stride=maxpool_stride),
            
            nn.Conv2d(hid2_size,hid3_size,k_conv_size, stride=conv_stride, padding=conv_pad),
            nn.BatchNorm2d(hid3_size),
            nn.ReLU(),
            
            nn.Conv2d(hid3_size,hid3_size,k_conv_size, stride=conv_stride, padding=conv_pad),
            nn.BatchNorm2d(hid3_size),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=maxpool_kernel,stride=maxpool_stride),
            
            nn.Conv2d(hid3_size,hid4_size,k_conv_size, stride=conv_stride, padding=conv_pad),
            nn.BatchNorm2d(hid4_size),
            nn.ReLU(),
            
            nn.Conv2d(hid4_size,hid4_size,k_conv_size, stride=conv_stride, padding=conv_pad),
            nn.BatchNorm2d(hid4_size),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=maxpool_kernel,stride=maxpool_stride),
            
            nn.Conv2d(hid4_size,hid4_size,k_conv_size, stride=conv_stride, padding=conv_pad),
            nn.BatchNorm2d(hid4_size),
            nn.ReLU(),
            
            nn.Conv2d(hid4_size,hid4_size,k_conv_size, stride=conv_stride, padding=conv_pad),
            nn.BatchNorm2d(hid4_size),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=maxpool_kernel,stride=maxpool_stride), 
            
            nn.Flatten(),
            
            nn.Linear(512, out_size)
            
        )
        
            
        
    def forward(self,x):
            out = self.convLayer(x)
            
            return out 

In [3]:
vgg_11 = VGG_11()
file1 = open("outputs.txt","w+")

In [4]:
# Unpickle a data item
def load_pickle(f):
    version = platform.python_version_tuple()
    if version[0] == '2':
        return  pickle.load(f)
    elif version[0] == '3':
        return  pickle.load(f, encoding='latin1')
    raise ValueError("invalid python version: {}".format(version))

In [5]:
# Load CIFAR-10 batch
def load_CIFAR_batch(filename):
    with open(filename, 'rb') as f:
        datadict = load_pickle(f)
        X = datadict['data']
        Y = datadict['labels']
        X = X.reshape(10000,3,32,32)
        Y = np.array(Y)
        return X, Y   

In [6]:
# Load full CIFAR-10 dataset
def load_CIFAR10(ROOT):
    xs = []
    ys = []
    for b in range(1,6):
        f = os.path.join(ROOT, 'data_batch_%d' % (b, ))
        X, Y = load_CIFAR_batch(f)
        xs.append(X)
        ys.append(Y)
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X, Y
    Xte, Yte = load_CIFAR_batch(os.path.join(ROOT, 'test_batch'))
    return Xtr, Ytr, Xte, Yte

In [7]:
def get_CIFAR10_data(num_training=49000, num_validation=0, num_test=10000):
    # Load the raw CIFAR-10 data
    dirname = os.path.dirname(__file__)
    cifar10_dir = 'CIFAR-10-DS/cifar-10-batches-py/'
    filename = os.path.join(dirname, cifar10_dir)
    
    X_train, y_train, X_test, y_test = load_CIFAR10(filename)

    num_training=X_train.shape[0]
    num_test=X_test.shape[0]

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    x_train = X_train.astype('float32')
    x_test = X_test.astype('float32')

    x_train /= 255
    x_test /= 255

    #Save dataset in npy files
    save('train_data.npy', x_train)
    save('train_label.npy', y_train)

    save('test_data.npy', x_test)
    save('test_label.npy', y_test)

    # Loading from npy files
    x_train = load('train_data.npy')
    y_train = load('train_label.npy')

    x_test = load('test_data.npy')
    y_test = load('test_label.npy')

    #Shuffle data 
    data = list(zip(x_train, y_train))
    np.random.shuffle(data)
    x_train, y_train = zip(*data)

    # Save the shuffled data
    save('train_data.npy', x_train)
    save('train_label.npy', y_train)

    #return x_train, y_train, x_test, y_test 

In [8]:
#Create custom size batches of the dataset
def createBatches(data,batch_size_required,device="cpu"):
    batch_size = int(len(data)/batch_size_required)
    res=[]

    #print('data ', data)
    for i in range (batch_size):
        batched_data = data[i*batch_size_required:i*batch_size_required+batch_size_required]
        res.append(batched_data)

    #print('CP 7 ', res)
    if (device == "cpu"):
        res = np.asarray(res)

    elif (device == "cuda"):
        print(type(res))
        #res = torch.FloatTensor(res)
        return res
    return res

In [9]:
def useLossFunction():
    criterion = nn.CrossEntropyLoss()
    return criterion

In [10]:
def useOptimizerFunction(name):
    learning_rate = 0.01                        # Learning rate
    momentum_val = 0.9                          # Momentum value

    optimizer=''
    if(name == 'Adadelta'):
        optimizer = optim.Adadelta(vgg_11.parameters())

    elif(name=='SGD'):
        optimizer = optim.SGD(vgg_11.parameters(), lr=learning_rate, momentum=momentum_val)
    
    elif(name=='NAG'):
        optimizer = optim.SGD(vgg_11.parameters(), lr=learning_rate, momentum=momentum_val, dampening=0, weight_decay=0, nesterov=True)

    return optimizer

In [19]:
def trainNetwork(max_epoch, x_train_tensor,y_train_tensor,optimizer,lossFun):
    for epoch in range(max_epoch):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(x_train_tensor, 0):

            inputs = data
            labels = y_train_tensor[i]
            optimizer.zero_grad()
            outputs = vgg_11(inputs)
            
            print('labels ', labels)
            print('labels shape', labels.size())

            
            loss = lossFun(outputs, labels
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            
            # print statistics        
            if i % 100 == 99:    # print every 100 mini-batches
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 100))
                
                msg = "loss is " + str(running_loss / 100) + " \n"
                file1.writelines(msg)

                running_loss = 0.0

    print('Finished Training')

SyntaxError: invalid syntax (<ipython-input-19-600d04a901ca>, line 17)

In [20]:
#Accuracy of individual classes and overall dataset
def AccuracyOfIndividualClassesAndDataset(x_test_t,y_test_t,bs):
    
    classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    correct = 0
    total = 0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    print('Class total before ', class_total)
    print('CP 6 ', x_test_t)

    with torch.no_grad():
        for i, data in enumerate(x_test_t, 0):
            #images, labels = data

            images = data
            labels = y_test_t[i]

            outputs = vgg_11(images)
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            c = (predicted == labels).squeeze()
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1


    for i in range(10):
        print('Class total after ', class_total)
        print('Accuracy of %5s : %.2f %%' % (
            classes[i], 100 * class_correct[i] / class_total[i]))
        msg = "'Accuracy of " + str(classes[i]) + "is " + str(100 * class_correct[i] / class_total[i]) + " \n"
        file1.writelines(msg)
    
    print('Accuracy of the network on the %d test images: %.2f %%' % (len(y_test_t)*bs,100 * correct / total))
    #temp = str((len(y_test_t)*bs,100 * correct / total)) + " \n"

In [21]:
#Main function
def main():
    max_epoch_num = 150               # Maximun numbe of epochs
    
    #Get train and test dataset
    #get_CIFAR10_data()


    # Load dataset from npy files
    x_train = load('train_data.npy')
    y_train = load('train_label.npy')

    x_test = load('test_data.npy')
    y_test = load('test_label.npy')
    
    #Divide the dataset into small batches
    batch_size = 64
    x_train = createBatches(x_train,batch_size)
    y_train = createBatches(y_train,batch_size)

    x_test = createBatches(x_test,batch_size)
    y_test = createBatches(y_test,batch_size)


    # Convert npArray to tensor
    x_train_tensor = torch.as_tensor(x_train)
    y_train_tensor = torch.as_tensor(y_train)

    x_test_tensor = torch.as_tensor(x_test)
    y_test_tensor = torch.as_tensor(y_test)

    criterion = useLossFunction()

    # Adadelta Optimizer
    optimizer=useOptimizerFunction('Adadelta')
    trainNetwork(max_epoch_num, x_train_tensor,y_train_tensor,optimizer,criterion)
    AccuracyOfIndividualClassesAndDataset(x_test_tensor,y_test_tensor,batch_size)
"""
    # SGD Optimizer
    optimizer=useOptimizerFunction('SGD')
    trainNetwork(max_epoch_num, x_train_tensor,y_train_tensor,optimizer,criterion)
    AccuracyOfIndividualClassesAndDataset(x_test_tensor,y_test_tensor,batch_size)

    # NAG optimzer
    optimizer=useOptimizerFunction('NAG')
    trainNetwork(max_epoch_num, x_train_tensor,y_train_tensor,optimizer,criterion)
    AccuracyOfIndividualClassesAndDataset(x_test_tensor,y_test_tensor,batch_size)


    print('Time required to run the  file', datetime.now() - begin_time)

    test2 = str ( (datetime.now() - begin_time) )
    file1.writelines( test2 )
"""


"\n    # SGD Optimizer\n    optimizer=useOptimizerFunction('SGD')\n    trainNetwork(max_epoch_num, x_train_tensor,y_train_tensor,optimizer,criterion)\n    AccuracyOfIndividualClassesAndDataset(x_test_tensor,y_test_tensor,batch_size)\n\n    # NAG optimzer\n    optimizer=useOptimizerFunction('NAG')\n    trainNetwork(max_epoch_num, x_train_tensor,y_train_tensor,optimizer,criterion)\n    AccuracyOfIndividualClassesAndDataset(x_test_tensor,y_test_tensor,batch_size)\n\n\n    print('Time required to run the  file', datetime.now() - begin_time)\n\n    test2 = str ( (datetime.now() - begin_time) )\n    file1.writelines( test2 )\n"

In [22]:
main()

labels  tensor([7, 2, 4, 6, 4, 6, 0, 3, 6, 2, 0, 8, 0, 3, 5, 8, 5, 9, 0, 9, 0, 2, 9, 4,
        5, 5, 1, 9, 9, 7, 2, 4, 2, 6, 6, 1, 2, 6, 4, 1, 0, 9, 5, 9, 5, 0, 0, 5,
        7, 8, 4, 4, 2, 9, 6, 5, 6, 3, 5, 7, 1, 1, 0, 6])
labels shape torch.Size([64])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [32]:
if __name__ == "__main__":
    # execute only if run as a script
    main()

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same

In [13]:
## Test code here
if (torch.cuda.is_available()):
        print(torch.cuda.get_device_name(torch.cuda.current_device()))
        cuda = torch.device('cuda')                                                     # Setting default CUDA device
        #score_pytorch_gpu = torch.empty((max_iteration,1),dtype=torch.float32)         # Tensor store on GPU, to conatain Xn and Xn+1
        #score_pytorch_gpu = logisticMapPyTorch(x_init, score_pytorch_gpu)              # Calling Logistic Map function with initial values
    
        x_train = load('./train_data.npy')
        x_train_gpu = torch.FloatTensor(x_train)

GeForce GTX 1060 6GB


In [17]:
print('CP 3',x_train_gpu.device)

CP 3 cuda:0


In [24]:
x_train_gpu = torch.FloatTensor(x_train).cuda()

y_train_gpu = torch.FloatTensor(load('train_label.npy')).cuda()

x_test_gpu = torch.FloatTensor(load('test_data.npy')).cuda()

y_test_gpu = torch.FloatTensor(load('test_label.npy')).cuda()

In [25]:
vgg_11 = vgg_11.cuda()

In [26]:
from datetime import datetime
begin_time = datetime.now()  

#Divide the dataset into small batches'
batch_size = 64
x_train_gpu = createBatches(x_train_gpu,batch_size, "cuda")
y_train_gpu = createBatches(y_train_gpu,batch_size, "cuda")

x_test_gpu = createBatches(x_test_gpu,batch_size, "cuda")
y_test_gpu = createBatches(y_test_gpu,batch_size, "cuda")

print('Time required to run the  file', datetime.now() - begin_time)

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
Time required to run the  file 0:00:00.008020


In [27]:
type(x_train_gpu)

list

In [28]:
begin_time = datetime.now()  

max_epoch_num = 150               # Maximun numbe of epochs
# Convert npArray to tensor
x_train_tensor =  (x_train_gpu)
#x_train_tensor =  torch.as_tensor(x_train_gpu)
# torch.as_tensor(x_train)
y_train_tensor = y_train_gpu

x_test_tensor = x_test_gpu
y_test_tensor = y_test_gpu

criterion = useLossFunction()

print('Time required to run the  file', datetime.now() - begin_time)

Time required to run the  file 0:00:00.000657


In [29]:
begin_time = datetime.now()  

# Adadelta Optimizer
optimizer=useOptimizerFunction('Adadelta')

x_train_tensor_long = x_train_tensor
trainNetwork(max_epoch_num, x_train_tensor_long,y_train_tensor,optimizer,criterion)

print('Time required to run the  file', datetime.now() - begin_time)

labels  tensor([7., 2., 4., 6., 4., 6., 0., 3., 6., 2., 0., 8., 0., 3., 5., 8., 5., 9.,
        0., 9., 0., 2., 9., 4., 5., 5., 1., 9., 9., 7., 2., 4., 2., 6., 6., 1.,
        2., 6., 4., 1., 0., 9., 5., 9., 5., 0., 0., 5., 7., 8., 4., 4., 2., 9.,
        6., 5., 6., 3., 5., 7., 1., 1., 0., 6.], device='cuda:0')
labels shape torch.Size([64])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [ ]:
AccuracyOfIndividualClassesAndDataset(x_test_tensor,y_test_tensor,batch_size)

In [ ]:
begin_time = datetime.now()  

# SGD Optimizer
optimizer=useOptimizerFunction('SGD')
trainNetwork(max_epoch_num, x_train_tensor,y_train_tensor,optimizer,criterion)
AccuracyOfIndividualClassesAndDataset(x_test_tensor,y_test_tensor,batch_size)

print('Time required to run the  file', datetime.now() - begin_time)

In [ ]:
begin_time = datetime.now()  

# NAG optimzer
optimizer=useOptimizerFunction('NAG')
trainNetwork(max_epoch_num, x_train_tensor,y_train_tensor,optimizer,criterion)
AccuracyOfIndividualClassesAndDataset(x_test_tensor,y_test_tensor,batch_size)


print('Time required to run the  file', datetime.now() - begin_time)

test2 = str ( (datetime.now() - begin_time) )